## Generative Adverserial Network Quiz

This notebook is contains guide and questions in order to complete the quiz. Plase mind that this quiz is semi-progressive, means that in order to answer future questions you might have to complete the previous one. 


#  <font color="red">**QUESTION 1**</font> 
GAN is a generative model. Please select all box that represents a generative models :
- [ ] Naive Bayes
- [ ] Logistic Regression
- [ ] Support Vector Machine
- [ ] K-Nearest Neighbor

#  <font color="red">**QUESTION 2**</font> 
What does adversarial refers to in GAN ? 
- [ ] Loss function
- [ ] Optimizer
- [ ] Input
- [ ] Output

#  <font color="red">**QUESTION 3**</font> 
Select all correct boxes  
- [ ] Generator takes input from Discriminator
- [ ] Discriminator tries to maximize the loss function
- [ ] Discriminator tries to generate a random noise 
- [ ] GAN input is random vector

Now, let's your comprehension in implementing GAN (DC GAN). These codes contains blank ("?" marks) that you should fill to complete and andswer the questions.

In [0]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense, Conv2D, Flatten, BatchNormalization, Dropout
from tensorflow.keras.layers import Reshape, UpSampling2D, MaxPooling2D, Activation
from tensorflow.keras.datasets import mnist, fashion_mnist
from tensorflow.keras.optimizers import SGD, RMSprop
from tensorflow.keras.utils import to_categorical

from PIL import Image

### Load MNIST Dataset
In this quiz, we will use MNIST Fashion dataset. Please write your code on the "??" mark

In [0]:
# Load mnist fashion data
(X_train, y_train), (X_test, y_test) = ?? 

# Do a basic normalization as shown in class
X_train = ??

X_train = X_train[:, :, :, None]
X_test = X_test[:, :, :, None]

In [0]:
print (X_train.shape, X_test.shape)


#  <font color="red">**QUESTION 4**</font> 
To ensure you have correctly load the data, what is the shape of X_train and X_test respectively?
- [ ] (60000, 28, 28, 1) and (10000, 28, 28, 1)
- [ ] (50000, 28, 28, 1) and (5000, 28, 28, 1)
- [ ] (60000, 32, 32, 1) and (10000, 32, 32, 1)
- [ ] (50000, 32, 32, 1) and (5000, 32, 32, 1)

#  <font color="red">**QUESTION 5**</font> 
While loading the Mnist dataset, in line 4, what does `X_train = (X_train.astype(np.float32) - 127.5)/127.5 ` means ?

- [ ] Rescale the data into within the interval of -1 to 1
- [ ] Rescale the data into within the interval of 0 to 1
- [ ] Denormalize data
- [ ] Reshape data




### Generator Model
Create a generator for your GAN

In [0]:
def generator_model():
    model = Sequential([
        Dense(1024, input_dim=100, activation='tanh'),
        Dense(128*7*7),
        BatchNormalization(),
        Activation('tanh'),

        # Do a reshape to make data shape into (7, 7, 128). Hint: Use Reshape()
        ??

        UpSampling2D(size=(2, 2)),
        Conv2D(64, (5, 5), padding='same', activation='tanh'),

        # Do upsampling with size (2,2). Hint: use UpSampling2D()
        ??

        # Complete the conv2D with total number of filter=1 and filter size=5x5 
        Conv2D(?, (?, ?), padding='same', activation='tanh')
    ])
    return model

In [0]:
generator_model().summary()

#  <font color="red">**QUESTION 6**</font> 
How many params does the generator_model has ? 
- [ ] 6,765,313
- [ ] 6,763,777
- [ ] 6,751,233
- [ ] 12,544

#  <font color="red">**QUESTION 7**</font> 
If you change the last convolution layer's filter shape into 7x7, what will be the final total parameters ?
- [ ] 6,765,313
- [ ] 6,763,777
- [ ] 6,741,233
- [ ] 12,700

### Discriminator Model
Create a discriminator for your GAN

In [0]:
def discriminator_model():
    model = Sequential([
        Conv2D(64, (5, 5), input_shape=(28, 28, 1), padding='same', activation='tanh'),
        
        # add max pooling with pool size 2x3
        ?? 
        Conv2D(128, (5, 5),activation='tanh'),
        MaxPooling2D(pool_size=(2, 2)),
        Flatten(),

        # Add Dense containing 1024 neurons with activation function ='tanh' 
        ?

        # Add single neuron (dense) with actiation='sigmoid'
        ?
    ])
    return model

In [0]:
discriminator_model().summary()

#  <font color="red">**QUESTION 8**</font> 
What does Flatten() layer means in the Discriminator?
- [ ] To make the data standardized 
- [ ] To make the data mean = 0
- [ ] To reshape the data into one dimensional vector
- [ ] To compile the previous layers

### GAN = Generator + Discriminator

In [0]:
def combine_model(g, d):
    model = Sequential()
    model.add(g)
    model.add(d)
    return model

## Training Functions
untuk melakukan training diperlukan BATCH_SIZE yang merupakan banyaknya gambar yang di train tiap epochs nya. Tiap piksel gambar akan dikonversikan menjadi nilai antara [-1,1). Hasil training akan disimpan pada file discriminator dan generator.

### Combine Generated Image

Combine Image adalah fungsi untuk menggabungkan gambar kedalam satu frame.

In [0]:
def combine_images(generated_images):
    num = generated_images.shape[0]
    width = int(np.sqrt(num))
    height = int(np.ceil(float(num)/width))
    shape = generated_images.shape[1:3]
    image = np.zeros((height*shape[0], width*shape[1]),
                     dtype=generated_images.dtype)
    for index, img in enumerate(generated_images):
        i = int(index/width)
        j = index % width
        image[i*shape[0]:(i+1)*shape[0], j*shape[1]:(j+1)*shape[1]] = \
            img[:, :, 0]
    return image

### Training Function

In [0]:
def train_gan(X_train, Y_train, batch_size, epochs, g, d, save_every=500, print_every=100):
    
    # ukuran vektor z
    z_size = g.layers[0].input_shape[1]
    
    # gabungkan Discriminator dan Generator
    d.trainable = False # set Discriminator tidak bisa dilatih sebelum digabung
    d_on_g = combine_model(g, d)    
    dg_optim = RMSprop (lr=0.0005)
    g_optim = RMSprop (lr=0.0005)
    d_on_g.compile(loss='binary_crossentropy', optimizer=dg_optim)
    
    g.compile(loss='binary_crossentropy', optimizer=g_optim)
    
    # set Discriminator agar bisa dilatih kembali
    d.trainable = True
    d_optim = RMSprop (lr=0.0005)
    d.compile(loss='binary_crossentropy', optimizer=d_optim)
    
    print("Number of batches", int(X_train.shape[0]/batch_size))
    
    # mulai pelatihan
    for epoch in range(epochs):
        print("\n-------------------------------\nEpoch :", epoch)        
        
        for index in range(int(X_train.shape[0]/batch_size)):
            
            # bangkitkan matrix z secara acak
            noise = np.random.uniform(-1, 1, size=(batch_size, z_size))
            
            # bangkitkan data gambar palsu dari matrix z
            generated_images = g.predict(noise, verbose=0)
            
            # ambil data gambar asli
            image_batch = X_train[index*batch_size:(index+1)*batch_size]
            
            if index % save_every == 0:
                image = combine_images(generated_images)
                image = image*127.5+127.5
                # Image.fromarray(image.astype(np.uint8)).save("train_ep"+
                #     str(epoch)+"_"+str(index)+".png")
                
                plt.imshow(image, cmap=plt.get_cmap('gray'))
                plt.axis('off')
                plt.show()
                
            # gabungkan data untuk pelatihan Discriminator
            X = np.concatenate((image_batch, generated_images))
            y = [1] * batch_size + [0] * batch_size
            
            # latih Discriminator
            d_loss = d.train_on_batch(X, y)           
            
            # bangkitkan matrix z secara acak untuk pelatihan Generator
            noise = np.random.uniform(-1, 1, (batch_size, z_size))
            
            # set Discriminator tidak bisa dilatih sebelum digabung
            d.trainable = False            
            
            # latih Generator
            g_loss = d_on_g.train_on_batch(noise, [1] * batch_size)
            
            # print loss
            if index % print_every == 0: 
                print("batch %d, g_loss : %f, d_loss : %f" % (index, g_loss, d_loss))
            
            # set Discriminator agar bisa dilatih kembali
            d.trainable = True       
            
        
    return g, d

## Training Process

### Initialize Model

In [0]:
z_size = 100
g_model = generator_model()
d_model = discriminator_model()

### Train GAN

In [0]:
batch = 225
epochs = 20
g_model, d_model = train_gan(X_train,y_train, batch, epochs, g_model, d_model)

## Generate image


In [0]:
seed = np.random.uniform(-1, 1, (4, 100))
images = g_model.predict(seed)

for i in range(4):
    plt.subplot(2,2,1+i)
    plt.imshow(np.reshape(images[i], (28,28,)),cmap=plt.get_cmap('gray'))
    plt.axis('off')
plt.show()

### Generate image from generator

In [0]:
def generate_images(g, batch_size):
    z_size = g.layers[0].input_shape[1]
    noise = np.random.uniform(-1, 1, (batch_size, z_size))
    generated_images = g.predict(noise, verbose=1)
    image = combine_images(generated_images)
    filename = "generated_image.png"
    image = image*127.5+127.5
    Image.fromarray(image.astype(np.uint8)).save(filename)
    return image

In [0]:
images = generate_images(g_model, 100)
plt.imshow(images, cmap=plt.get_cmap('gray'))
plt.axis('off')
plt.show()

### Generate image with the check of discriminator

In [0]:
def generate_best_images(g, d, batch_size):
    z_size = g.layers[0].input_shape[1]
    noise = np.random.uniform(-1, 1, (batch_size*20, z_size))

    generated_images = g.predict(noise, verbose=1)
    d_pret = d.predict(generated_images, verbose=1)

    index = np.arange(0, batch_size*20)
    index.resize((batch_size*20, 1))

    pre_with_index = list(np.append(d_pret, index, axis=1))
    pre_with_index.sort(key=lambda x: x[0], reverse=True)

    nice_images = np.zeros((batch_size,) + generated_images.shape[1:3], dtype=np.float32)
    nice_images = nice_images[:, :, :, None]

    for i in range(batch_size):
        idx = int(pre_with_index[i][1])
        nice_images[i, :, :, 0] = generated_images[idx, :, :, 0]

    image = combine_images(nice_images)
    filename = "generated_image_best.png"
    image = image*127.5+127.5
    Image.fromarray(image.astype(np.uint8)).save(filename)
    return image

In [0]:
images = generate_best_images(g_model, d_model, 100)
plt.imshow(images, cmap=plt.get_cmap('gray'))
plt.axis('off')
plt.show()

#  <font color="red">**QUESTION 9**</font> 
what is the differences between images that are not checked by discriminator (directly generated from generator), and images that previously checked by discriminator?
- [ ] The unchecked images are smaller in size
- [ ] The checked images are usually better 
- [ ] The unchecked images are usually better
- [ ] The checked images are smaller in size

#  <font color="red">**QUESTION 10**</font> 
What do you think about losses over time while training ? 
- [ ] It will be minimized overtime
- [ ] It will be maximized overtime
- [ ] It will just fluctuate randomly